# Single indicator calibration

In [13]:
%run calibration_Cube_v4.py
%matplotlib

Using matplotlib backend: Qt5Agg


#### individual indicator fit

In [45]:
# initial parameter required
# additional label for RoI separated by space letter!!!
name_dyes = ['Pt-TPTBP+MY 1', 'Pt-TPTBP+MY 2', 'Pt-TPTBP+MY 3',
             'Pd-TPTBP+MY 1', 'Pd-TPTBP+MY 2', 'Pd-TPTBP+MY 3']
# !!! same order as name_dyes!!!!
name_singles = ['Pt-TPTBP+MY', 'Pd-TPTBP+MY']
analyte = 'O2'
unit = 'hPa' #%air or hPa

# pixel - selection from image cube shape: px(y/x) = HxW - anti-clockwise n the same order 
# as name_dyes
# TPTBP
pixel_rot = [[(850, 1530), (900, 1530), (900, 1580), (850, 1580)],
             [(910, 1500), (960, 1500), (960, 1580), (910, 1580)],
             [(1015, 1510), (1065, 1510), (1065, 1570), (1015, 1570)],
             # Pd-TPTBP
             [(750, 560), (800, 560), (800, 690), (750, 690)],
             [(850, 570), (930, 570), (930, 680), (850, 680)],
             [(960, 600), (1060, 600), (1060, 670), (960, 670)]]
# TFPP
#pixel_rot = [[(380, 570), (420, 570), (420, 650), (380, 650)],
#             [(450, 595), (540, 595), (540, 666), (450, 666)],
#             [(550, 595), (600, 595), (600, 660), (550, 660)],
             # Pd-TPTBP
#            [(380, 1465), (420, 1465), (420, 1528), (380, 1528)],
#            [(430, 1465), (490, 1465), (490, 1528), (430, 1528)],
#            [(500, 1470), (540, 1470), (540, 1528), (500, 1528)]]

# ----------------------------------
arg = {'px color': '#a40019', 'rotation': 180., 'show bands': 123, 'color Pt': '#ff8c00', 
       'color Pd': '#347474', 'figure size meas': (5,3), 'fontsize meas': 13., 'colors':'viridis'}

# individual/dual/all, reference, middle (meaning the rest)
what_fit = ['individual', 'reference', 'middle']
arg_fit = {'fit range sensor': (650, 900), 'fit range ref': (490, 580),
           'fit concentration': (0, 1000)}

#save_genic = True
save_op = {'type': ['png', 'tiff', 'svg'], 'dpi': 300.}

file_hdr = '20200701_ID-SZ10-16_radiometric/TPTBP/calibration/Optode_100ms_0hPa_cube_radiometric.hdr'
path_calib = '20200701_ID-SZ10-16_radiometric/TPTBP/calibration/' 
path_res = '20200701_ID-SZ10-16_radiometric/TPTBP/output/'

#### pre-test for pixel and individual file 

In [46]:
# test plot cube -> check whether selected pixel/area is within cube shape
ls_px = []
if pixel_rot:
    for p in pixel_rot:
        for pp in p:
            ls_px.append(pp)
else:
    ls_px = None
corr.load_testcube(file_hdr=file_hdr, pixel_rot=ls_px, span_area=True, arg=arg)      

#### determine generic function of single indicator

In [47]:
arg_fit['fit concentration'] =  (0, 60)
# =================================================================
start_time = time.time()
[dic_cube, dic_fitting, fig, 
 ax] = cC.generic_function_cube_v2(path_100=path_calib, pixel_rot=pixel_rot, arg=arg, arg_fit=arg_fit, 
                                   name_dyes=name_dyes, ls_sensor_fit=name_singles, path_res=path_res,
                                   what_fit=what_fit, unit=unit, analyte=analyte, plot_meas=False, 
                                   plot_result=True, saving=False)

print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))

Time of execution --- 6.300626993179321 seconds 


#### spectral deviation of generic function to plate-reader 

In [48]:
file_sensorID = 'C:/Users/au652733/OneDrive - Aarhus Universitet/Project organization/01 protocol/Sensors_ID.xlsx'

path_pr = "U:/Documents/UPython/09_optode_characterization/measurement/"
# TFPP
#file_ref1 = path_pr + '20-06-08 18-56-32_SZ-13_SZ-16_em420-840_ex390_anoxic.txt'
#file_ref2 = path_pr + '20-06-08 19-02-40_SZ-14_SZ-15_em440-750_ex410_anoxic.txt'
#name_foile = ['SZ-13', 'SZ-14']

# TPTBP
file_ref1 = path_pr + '20-06-08 18-16-10_SZ-4_SZ-6_SZ-7.2_SZ-7.3_SZ-9.3_SZ-9.2_SZ-11.3_SZ-12.3_em465-840_ex440_anoxic.txt'
file_ref2 = path_pr + '20-06-08 18-16-10_SZ-4_SZ-6_SZ-7.2_SZ-10.3_SZ-9.3_SZ-9.2_SZ-7.3_SZ-12.3_em465-840_ex440_anoxic.txt'
name_foile = ['SZ-11.3', 'SZ-10.3']

file_ref = [file_ref1, file_ref2]

In [49]:
corr.spectral_deviation(file_sensorID=file_sensorID, data_cube=dic_fitting['result'], 
                        file_ref=file_ref, name_foile=name_foile, arg=arg, arg_fit=arg_fit)

#### Split all calibration cubes into RoI and store information

In [36]:
# calibration of individual measurement points
arg_fit['fit concentration'] = (0, 1000)

# ==========================================================================
start_time = time.time()
fit_range = np.arange(arg_fit['fit concentration'][0], arg_fit['fit concentration'][1]+1)
dic_cube = dict()
for i in glob(path_calib + '/*.hdr'):
    if np.float(i.split(unit)[0].split('_')[-1]) in fit_range:
        cube, fig, ax = corr.hyperCube_preparation(i, arg=arg, name_dyes=name_dyes, unit=unit,
                                                   pixel_rot=pixel_rot, cube_type='single',
                                                   save=True, averaging=True, plotting=True)
        dic_cube[cube['Concentration']] = cube['region of interest']
print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))

Time of execution --- 42.98277831077576 seconds 


#### calibration single indicator

In [50]:
# stored calibration files for single indicators as hdf5 - Files (already corrected)
path_calib = '20200701_ID-SZ10-16_radiometric/TPTBP/output_1/correctionCube/singleIndicator/'
path_firesting = '20200701_ID-SZ10-16_radiometric/TPTBP/Firesting/O2_calibration_100ms.txt'

arg_fit['fit concentration'] = (0, 1000)
ls_outlier = None # provid a list with the concentration noted in the file name or state None ['50hPa']
#par0 = dict({'Pt-TFPP+MY': dict({'k': 0.0447, 'f': 0.887}), 
#             'Pd-TFPP+MY': dict({'k': 0.2132, 'f': 0.887})})
par0 = dict({'Pt-TPTBP+MY': dict({'k': 0.0447, 'f': 0.887}), 
             'Pd-TPTBP+MY': dict({'k': 0.2132, 'f': 0.887})})
arg['lambda reference'] = 500.

# ==================================================================
start_time = time.time()
%run calibration_Cube_v4.py

[dic_SVFit, calib_info, df_calib,
 dic_min] = singleSensor_calibration(path_calib=path_calib, arg=arg, arg_fit=arg_fit, par0=par0,
                                     what_fit=what_fit, name_singles=name_singles, name_RoI=name_dyes, 
                                     save_op=save_op, path_firesting=path_firesting, unit=unit,
                                     ls_outlier=ls_outlier, analyte=analyte, ratiometric=True, 
                                     plotting=True, save=False, simply=True)
print('Calibration check')
print('\t K(sv) -', name_singles[0], round(dic_SVFit[name_singles[0] + ' Fit']['Report'].best_values['k'],3))
print('\t K(sv) -', name_singles[1], round(dic_SVFit[name_singles[1] + ' Fit']['Report'].best_values['k'], 3))
print('\t K(sv) ratio', [x for x in name_singles if x.startswith('Pd')][0], '/', [x for x in name_singles if x.startswith('Pt')][0], 
      round(dic_SVFit[[x for x in name_singles if x.startswith('Pd')][0] + ' Fit']['Report'].best_values['k'] / dic_SVFit[[x for x in name_singles if x.startswith('Pt')][0] + ' Fit']['Report'].best_values['k'], 3))
print('=================================')
print("Time of execution --- %s seconds " % (time.time() - start_time))


integral
Calibration check
	 K(sv) - Pt-TPTBP+MY 0.016
	 K(sv) - Pd-TPTBP+MY 0.054
	 K(sv) ratio Pd-TPTBP+MY / Pt-TPTBP+MY 3.372
Time of execution --- 9.773608207702637 seconds 


In [38]:
import seaborn as sns
fs = 11
colors = dict({name_dyes[0]: sns.cubehelix_palette(len(dic_min.keys()), start=.5, rot=-.75), 
              name_dyes[1]: sns.cubehelix_palette(len(dic_min.keys()))})
fig1, ax1 = plt.subplots(figsize=(7,3), ncols=2, sharex=True)
axR1 = ax1[0].twinx()
axT1 = ax1[0].twiny()
axR1.get_shared_y_axes().join(axR1, ax1[0])
axT1.get_shared_x_axes().join(axT1, ax1[0])
axR2 = ax1[1].twinx()
axT2 = ax1[1].twiny()
axR2.get_shared_y_axes().join(axR2, ax1[1])
axT2.get_shared_x_axes().join(axT2, ax1[1])

ax1[0].set_title(name_singles[0], fontsize=fs*0.8, loc='left')
ax1[1].set_title(name_singles[1], fontsize=fs*0.8, loc='left')

df_sens1 = pd.concat([pd.concat(dic_min[c], axis=1).filter(like=name_singles[0]).mean(axis=1) 
                      for c in dic_min.keys()], axis=1, keys=dic_min.keys())
df_sens2 = pd.concat([pd.concat(dic_min[c], axis=1).filter(like=name_singles[1]).mean(axis=1) 
                      for c in dic_min.keys()], axis=1, keys=dic_min.keys())
sens1_max = max([dic_min[c][name_dyes[0]].loc[650:].max().values[0] for c in dic_min.keys()])
sens2_max = max([dic_min[c][name_dyes[1]].loc[650:].max().values[0] for c in dic_min.keys()])
for en, c in enumerate(dic_min.keys()):
    ax1[0].plot(df_sens1[c]/sens1_max, color=colors[name_dyes[0]][en])
    ax1[1].plot(df_sens2[c]/sens2_max, color=colors[name_dyes[1]][en])

ax1[0].tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8)
axR1.tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8, labelcolor='white')
axT1.tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8, labelcolor='white')
ax1[1].tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8)
axR2.tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8, labelcolor='white')
axT2.tick_params(axis='both', which='both', direction='in', labelsize=fs*0.8, labelcolor='white')

ax1[0].set_ylabel('Rel. fluorescence [rfu]', fontsize=fs) 
ax1[1].set_ylabel('Rel. fluorescence [rfu]', fontsize=fs) 
ax1[0].set_xlabel('Wavelength [nm]', fontsize=fs)
ax1[1].set_xlabel('Wavelength [nm]', fontsize=fs)
plt.tight_layout()
plt.show()